In [2]:
pip install pygame numpy

  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/c8/db/db5a6567791d26d610b2c25cbf74be527857b6dff68e566d57bd7954ece6/pygame-2.5.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 6.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [58]:
import os
import pygame
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np

####

import sys
from io import StringIO

captured_output = StringIO()
sys.stdout = captured_outp


####

class ProgramaEstimulo:
    def __init__(self, root):
        self.root = root
        self.sounds_folder = "sonidos"  
        self.images_folder = "imagenes" 

        self.instrucciones_label = tk.Label(root, text="Va a escuchar una serie de palabras. Cada palabra denomina a una de las dos figuras que aparecen. Elija la figura (1 ó 2) haciendo clic con el ratón en la imagen correspondiente.")
        self.instrucciones_label.pack()

        self.boton_comenzar = tk.Button(root, text="Comenzar", command=self.iniciar_presentacion)
        self.boton_comenzar.pack()

        self.boton_pausa = tk.Button(root, text="Pausar", command=self.pausar_presentacion, state=tk.DISABLED)
        self.boton_pausa.pack()

        self.presentacion_activa = False
        self.pausa_activa = False
        self.opcion_elegida = None
        self.sonidos = [os.path.join(self.sounds_folder, f"sonido_{i}.wav") for i in range(1, 41)]
        self.indice_actual = 0

    def iniciar_presentacion(self):
        self.instrucciones_label.pack_forget()
        self.boton_comenzar.pack_forget()

        self.presentacion_activa = True

        # Configurar la ventana para presentar estímulos
        self.canvas = tk.Canvas(self.root, width=600, height=400)
        self.canvas.pack()

        self.boton_pausa.config(state=tk.NORMAL)
        self.mostrar_estimulo()

    def pausar_presentacion(self):
        if self.presentacion_activa:
            self.pausa_activa = not self.pausa_activa
            if self.pausa_activa:
                self.boton_pausa.config(text="Reanudar")
            else:
                self.boton_pausa.config(text="Pausar")
                self.mostrar_estimulo()  # Reanudar la presentación

    def mostrar_estimulo(self):
        if not self.pausa_activa and self.presentacion_activa and self.indice_actual < len(self.sonidos):
            self.mostrar_ruido()

    def mostrar_ruido(self):
        ruido = np.random.normal(128, 50, (400, 600)).astype(np.uint8)
        imagen_ruido = Image.fromarray(ruido, mode='L')
        self.imagen_ruido_tk = ImageTk.PhotoImage(imagen_ruido)

        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.imagen_ruido_tk)
        self.root.update()
        self.root.after(1000, self.mostrar_cruz)

    def mostrar_cruz(self):
        # cruz pequeña después de desaparecer el ruido
        self.canvas.delete("all")
        self.canvas.create_line(300 - 5, 200, 300 + 5, 200, fill="black", width=2)  # Línea horizontal
        self.canvas.create_line(300, 200 - 5, 300, 200 + 5, fill="black", width=2)  # Línea vertical
        self.root.update()
        self.root.after(1000, self.reproducir_sonido)

    def reproducir_sonido(self):
        # Reproducir sonido
        pygame.mixer.init()
        pygame.mixer.music.load(self.sonidos[self.indice_actual])
        pygame.mixer.music.play()

        # Esperar a que termine la reproducción del sonido antes de mostrar opciones
        self.root.after(int(pygame.mixer.Sound(self.sonidos[self.indice_actual]).get_length() * 1000), self.mostrar_imagenes_opciones)

    def mostrar_imagenes_opciones(self):
        # Eliminar cruz y mostrar opciones
        self.canvas.delete("all")

        # Cargar imágenes
        imagen_1_path = os.path.join(self.images_folder, f"imagen_{self.indice_actual + 1}_1.png")
        imagen_2_path = os.path.join(self.images_folder, f"imagen_{self.indice_actual + 1}_2.png")

        imagen_1 = Image.open(imagen_1_path)
        imagen_2 = Image.open(imagen_2_path)

        imagen_1 = imagen_1.resize((200, 200))
        imagen_2 = imagen_2.resize((200, 200))

        self.imagen_1_tk = ImageTk.PhotoImage(imagen_1)
        self.imagen_2_tk = ImageTk.PhotoImage(imagen_2)

        # Respuesta con clic del ratón
        self.canvas.create_image(50, 100, anchor=tk.NW, image=self.imagen_1_tk, tags="imagen_1")
        self.canvas.create_image(350, 100, anchor=tk.NW, image=self.imagen_2_tk, tags="imagen_2")

        self.canvas.tag_bind("imagen_1", "<Button-1>", lambda event: self.registrar_respuesta(1))
        self.canvas.tag_bind("imagen_2", "<Button-1>", lambda event: self.registrar_respuesta(2))

    def registrar_respuesta(self, opcion_elegida):
        print(f"Respuesta del usuario para el sonido {self.indice_actual + 1}: {opcion_elegida}")

        # Limpieza y preparación para el próximo estímulo
        self.indice_actual += 1

        # Si se han presentado todos los estímulos, mostrar el mensaje de agradecimiento
        if self.indice_actual == len(self.sonidos):
            self.presentacion_activa = False
            self.mostrar_agradecimiento()
        else:
            self.mostrar_estimulo()

    def mostrar_agradecimiento(self):
        self.canvas.delete("all")
        mensaje_agradecimiento = tk.Label(self.root, text="Muchas gracias por participar en este experimento.")
        mensaje_agradecimiento.config(font=("Helvetica", 16))
        mensaje_agradecimiento.pack(side=tk.TOP, padx=20, pady=20)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Práctica 1: El efecto bouba/kiki. Wolfgang Köhler (1929)")

    programa = ProgramaEstimulo(root)

    root.mainloop()
    
    
    
##
# Restaurar la salida estándar
sys.stdout = sys.__stdout__

# Obtener la salida capturada como una cadena
output_str = captured_output.getvalue()
##


with open("resultados.txt", "w") as file:
    file.write(output_str)


Respuesta del usuario para el sonido 1: 1
Respuesta del usuario para el sonido 2: 1
Respuesta del usuario para el sonido 3: 1
